# Monte Carlo Simulation

In [ ]:
### import necessary modules
import math
import json
import numpy as np
from _module import mc
from matplotlib import pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
%config InlineBackend.figure_format = 'svg'


### set parameters
L = 8 # linear size of the system
J = 1. # coupling strength
T_list = np.linspace(1., 3.54, 10) # list of temperatures
T_num = len(T_list)
mct = L**2 # monte carlo time
mc_eqSweeps = 10000 # num of monte carlo times to reach the equilibrium state
num_sample = 100000 # num of sampling
sample_store = [] # used to store samples


### monte carlo simulations for different temperatures
for temp in T_list:
    machine = mc.IsingLattice(L, 'random', temp, J)
    print("\nI'm finding the detailed balance for temperature: " + str(temp) + '.\n')
    for i in range(mct*mc_eqSweeps):
        machine.flip_spin()
    print("Start to sample at temperature " + str(temp) + ' -_-')
    for i in range(num_sample):
        for j in range(mct):
            machine.flip_spin()
        sample_store.append(list(np.reshape(machine.configuration[1:-1, 1:-1], L**2)))
    print("End sampling for temperature " + str(temp) + ' ^_^')


### save the data sampled from MC
print("\nGet Total " + str(len(sample_store)) + " configurations! Data is saving, please wait! @_@")
save_mc_data = [sample_store, T_list.tolist(), num_sample]
with open('data/mc_data.json', 'w') as mc_file:
    json.dump(save_mc_data, mc_file)
print("Done!")

In [ ]:
### visualization
bw_cmap = colors.ListedColormap(['black', 'white'])
plt.imshow(np.reshape(np.array(sample_store[-1]),[L,L]), cmap=bw_cmap)

# Restricted Boltzmann Machine

In [ ]:
### import necessary modules
import math
import json
import numpy as np
from _module import rbm


### set parameters
L = 8 # linear size of the system
v_num = L**2 # num of visible units
h_num = 4 # num of hidden units
batch_size = 50 # mini batch size
epoch = 100
lr = 0.01 # learning rate
cd_order = 20 # order of contrastive divergence
gs_eqSweeps = 10000 # num of Gibbs steps to reach the equilibrium state
num_sample = 100000 # num of sampling
sample_store = [] # used to store samples


### prepare training data set
print("I'm loading data!!!\t -_-")
with open('data/mc_data.json', 'r') as mc_file: # load the data sampled from monte carlo simulation
    [mc_data, T_list, _] = json.load(mc_file)
train_data = np.array_split(mc_data, len(T_list)) # split the data set according to the temperature
train_data = 0.5 * (np.array(train_data) + 1.) # map {-1,1} to {0,1}
T_iter = iter(T_list)


### RBM training and sampling
for data in train_data:
    print("\nRBM Start For Temperature: " + str(next(T_iter)) + '\n')
    model = rbm.Rbm(v_num, h_num)
    model.train(data, batch_size, epoch, lr, cd_order)
    rbm_sample_data = model.sample(np.random.choice([0., 1.], v_num), gs_eqSweeps, num_sample, 1)
    sample_store.append(rbm_sample_data.tolist())

    
### save the data sampled from RBM
print("\nSaving RBM's Sampling data! >_<\n")
save_rbm_data = [sample_store, T_list]
with open('data/rbm_data.json', 'w') as rbm_file:
    json.dump(save_rbm_data, rbm_file)
print("Done! @_@")

# Magnetism

In [ ]:
### import necessary modules
import math
import json
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
%config InlineBackend.figure_format = 'svg'


### function: get the average magnetism for a bunch of data
def mag_temp(x):
    m = list(map(lambda y: abs(np.mean(y)), x))
    m = np.sum(m)/len(m)
    return m


### load configurations sampled form MC & RBM
with open('data/mc_data.json', 'r') as mc_file:
    [mc_data, T_list, _] = json.load(mc_file)
with open('data/rbm_data.json', 'r') as rbm_file:
    [rbm_data, _] = json.load(rbm_file)


### calculate the average magnetism for each Temperature
mc_plot_data = np.array_split(mc_data, len(T_list))
mc_plot_data = list(map(mag_temp, mc_plot_data))
rbm_plot_data = 2. * np.array(rbm_data) - 1. # map {0,1} to {-1,1}
rbm_plot_data = list(map(mag_temp, rbm_plot_data))


### visualization
plt.plot(T_list, mc_plot_data, c='b', marker="+", label="Monte Carlo", linewidth=1.5, markersize=7)
plt.plot(T_list, rbm_plot_data, c='r', marker="x", label="RBM", linewidth=1.5, markersize=7)
plt.title("Magnetism From Monte Carlo and RBM")
plt.xlabel("Temperature")
plt.ylabel("Magnetism")
plt.xlim((1., 3.54))
plt.legend(loc='best')
fig = plt.gcf() # get current figure
fig.savefig('magnetism-h4.eps', format='eps', dpi=1000)